# Setting up environment

Check cuda version

In [1]:
!nvidia-smi

Thu Apr  3 07:09:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Clone GitHub repo

In [2]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 348, done.
remote: Counting objects: 100% (348/348), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 348 (delta 83), reused 146 (delta 44), pack-reused 0 (from 0)
Receiving objects: 100% (348/348), 9.53 MiB | 32.95 MiB/s, done.
Resolving deltas: 100% (83/83), done.


Change directory

In [29]:
%cd LLaMA-Factory

/kaggle/working/LLaMA-Factory


Install packages

In [4]:
!pip install -e ".[torch,metrics]"
!pip install deepspeed triton
!pip install flash-attn --no-build-isolation

Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 39.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 109.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109

# Loading Dataset

Import packages

In [26]:
from datasets import load_dataset, concatenate_datasets, Value, Dataset
import json

Load videos mapping

In [6]:
with open("/kaggle/input/d/seanjeanmoey/next-qa-dataset/map_vid_vidorID.json") as file:
    video_dir_map = json.load(file)

Format data

In [40]:
def format_data(sample):
    return {
        "messages": [
            {
                "content": f"<video>{sample['question']}",
                "role": "user"
            },
            {
                "content": f"{sample['answer']}",
                "role": "assistant"
            }
        ],
        "videos": [
            f"/kaggle/input/d/seanjeanmoey/next-qa-dataset/NExTVideo/NExTVideo/{video_dir_map[sample['video']]}.mp4"
        ]
    }

Format MCQ

In [8]:
def reformat_mcq(sample):
    choice_labels = ["A", "B", "C", "D", "E"]
    choices = [sample[f"a{i}"] for i in range(5)]
    formatted_choices = "\n".join([f"{choice_labels[i]}. {choice}" for i, choice in enumerate(choices)])
    
    return {
        "video": sample["video"],
        "frame_count": sample["frame_count"],
        "width": sample["width"],
        "height": sample["height"],
        "question": f"{sample['question']}\n{formatted_choices}\nSelect one best answer to the above multiple-choice question based on the video. Respond with only the letter (A, B, C, D or E) of the correct option.",
        "answer": choice_labels[sample["answer"]],
        "qid": sample["qid"],
        "type": sample["type"],
        "additional_ref_answer": None
    }

Load dataset

In [41]:
dataset_id = 'lmms-lab/NExTQA'

mcq_dataset = load_dataset(dataset_id, 'MC')['test'].map(reformat_mcq, remove_columns=['a0', 'a1', 'a2', 'a3', 'a4'])
new_features = mcq_dataset.features.copy()
new_features["video"] = Value("string")
new_features["frame_count"] = Value("int32")
new_features["width"] = Value("int32")
new_features["height"] = Value("int32")
new_features["qid"] = Value("int32")
mcq_dataset = mcq_dataset.cast(new_features)
train_test_split = mcq_dataset.train_test_split(test_size=0.3, seed=42)
val_test_split = train_test_split['test'].train_test_split(test_size=2/3, seed=42)
mcq_train_dataset = train_test_split['train']
mcq_eval_dataset = val_test_split['train']
mcq_test_dataset = val_test_split['test']

oe_train_dataset, oe_eval_dataset, oe_test_dataset = load_dataset(dataset_id, 'OE', split=['train', 'validation', 'test'])

train_dataset = concatenate_datasets([mcq_train_dataset, oe_train_dataset])
eval_dataset = concatenate_datasets([mcq_eval_dataset, oe_eval_dataset])
test_dataset = concatenate_datasets([mcq_test_dataset, oe_test_dataset])

train_dataset = [format_data(sample) for sample in train_dataset]
eval_dataset = [format_data(sample) for sample in eval_dataset]
test_dataset = [format_data(sample) for sample in test_dataset]

dataset = train_dataset + eval_dataset + test_dataset

Create dataset info json

In [42]:
args = { 
    "nextqa": {
        "file_name": "/kaggle/working/LLaMA-Factory/data/nextqa.json",
        "formatting": "sharegpt",
        "columns": {
            "messages": "messages",
            "videos": "videos"
        },
        "tags": {
            "role_tag": "role",
            "content_tag": "content",
            "user_tag": "user",
            "assistant_tag": "assistant"
        }
    }
}
with open("data/dataset_info.json", "w", encoding="utf-8") as f: 
    json.dump(args, f, ensure_ascii=False, indent=4)

Create dataset json

In [43]:
with open("data/nextqa.json", "w", encoding="utf-8") as f: 
    json.dump(dataset, f, ensure_ascii=False, indent=4)

# Fine-tuning Model

Import packages

In [12]:
import wandb
from kaggle_secrets import UserSecretsClient

Login to wandb

In [13]:
wandb.login(key=UserSecretsClient().get_secret("WANDB_API_KEY"))

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: seanjeanmoey123 (seanjeanmoey123-nus). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Create train.json

In [44]:
args = {
    "model_name_or_path": "Qwen/Qwen2.5-VL-3B-Instruct",
    "image_max_pixels": 262144,
    "video_max_pixels": 16384,
    "trust_remote_code": True,
    "stage": "sft",
    "do_train": True,
    "finetuning_type": "lora",
    "flash_attn": "auto",
    "lora_rank": 8,
    "lora_alpha": 16,
    "lora_dropout": 0,
    "lora_target": "all",
    "dataset_dir": "/kaggle/working/LLaMA-Factory/data",
    "dataset": "nextqa",
    "template": "qwen2_vl",
    "overwrite_cache": True,
    # "cutoff_len": 2048,
    # "max_samples": 128,
    # "preprocessing_num_workers": 16,
    "dataloader_num_workers": 4,
    "output_dir": "/kaggle/working/finetuned",
    "logging_steps": 5,
    "save_steps": 100,
    "plot_loss": True,
    "overwrite_output_dir": True,
    "save_only_model": False,
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps": 8,
    "learning_rate": 5.0e-5,
    "num_train_epochs": 3.0,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.1,
    "bf16": True,
    "ddp_timeout": 180000000,
    "resume_from_checkpoint": None,
    "max_grad_norm": 1,
    "warmup_steps": 0,
    "packing": False,
    "report_to": None,
    "optim": "adamw_torch",
    "streaming": True,
    "max_steps": 10000,
}
with open("train.json", "w", encoding="utf-8") as f: 
    json.dump(args, f, ensure_ascii=False, indent=4)

Train model

In [ ]:
!llamafactory-cli train train.json

2025-04-03 07:28:03.795557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 07:28:03.819062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-03 07:28:03.825818: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[2025-04-03 07:28:07,294] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[INFO|2025-04-03 07:28:11] llamafactory.cli:143 >> Initializing 2 distributed tasks at: 127.0.0.1:54861
W0403 07:28:13.539000 1500 torch/distributed/run.py:793] 
W0403 07:28:13.539000 1500 torch/distributed/run.py:793] **************************

# Merging Fine-tuned Model

Import packages

In [17]:
import json

Create merging script

In [18]:
args = {
    "model_name_or_path": "Qwen/Qwen2.5-VL-3B-Instruct",
    "adapter_name_or_path": "/kaggle/working/finetuned",
    "template": "qwen2_vl",
    "finetuning_type": "lora",
    "trust_remote_code": True,
    "export_dir": "/kaggle/working/merged",
    "export_size": 5,
    "export_device": "cpu",
    "export_legacy_format": False,
}
with open("merge.json", "w", encoding="utf-8") as f: 
    json.dump(args, f, ensure_ascii=False, indent=4)

Merge model

In [19]:
!llamafactory-cli export merge.json

^C
Traceback (most recent call last):
  File "/usr/local/bin/llamafactory-cli", line 5, in <module>
    from llamafactory.cli import main
  File "/kaggle/working/LLaMA-Factory/src/llamafactory/__init__.py", line 43, in <module>
    from .extras.env import VERSION
  File "/kaggle/working/LLaMA-Factory/src/llamafactory/extras/env.py", line 22, in <module>
    import peft
  File "/usr/local/lib/python3.10/dist-packages/peft/__init__.py", line 22, in <module>
    from .auto import (
  File "/usr/local/lib/python3.10/dist-packages/peft/auto.py", line 21, in <module>
    from transformers import (
  File "<frozen importlib._bootstrap>", line 1075, in _handle_fromlist
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py", line 1957, in __getattr__
    value = getattr(module, name)
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py", line 1956, in __getattr__
    module = self._get_module(self._class_to_module[name])
  File "/usr/lo

Zip the output

In [20]:
%cd /kaggle/working
!7z a -r finetuned.zip finetuned
!7z a -r merged.zip merged

/kaggle/working

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive:
  0M Sca        0 files, 0 bytes

Creating archive: finetuned.zip

Items to compress: 0

    
Files read from disk: 0
Archive size: 22 bytes (1 KiB)
Everything is Ok

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive:
  0M Sca        0 files, 0 bytes

Creating archive: merged.zip

Items to compress: 0

    
Files read from disk: 0
Archive size: 22 bytes (1 KiB)
Everything is Ok
